In [2]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from IPython.core.pylabtools import figsize
from IPython.display import IFrame
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix
from sklearn import datasets
from sklearn import metrics
from sklearn import neighbors                           
from sklearn import tree, linear_model,neighbors   
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.datasets import load_boston
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import MissingIndicator
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import*
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
#Preparing for data, I have saved the prepared data. Please skip this chunk.
sales_with_category = pd.read_csv("sales_with_category.csv")
sales_with_category.info()
sales_category_list_cust_date_PA = sales_with_category[sales_with_category['StateProvince']=='PA'].groupby(['CustomerName','Sale Date','Base.Implement','EquipmentCategoryDesc']).size()
cust_purchase_history_date = sales_category_list_cust_date_PA.to_frame().unstack().unstack()
cust_purchase_history_date1 = cust_purchase_history_date.dropna(axis = 1,how = 'all').sort_index(level = 2,axis = 1)
cust_purchase_history_date1.info()
cust_purchase_history_date1
cust_purchase_history_date1.to_csv('associate_rule_with_date&category_PA.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (70,77,80,84) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55377 entries, 0 to 55376
Columns: 107 entries, Unnamed: 0 to sold_to_code_y
dtypes: float64(20), int64(2), object(85)
memory usage: 45.2+ MB
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10699 entries, ('#NAME? ROBERT FITCH', '05/20/2019') to ('zakostelecky chris', '10/29/2020')
Data columns (total 35 columns):
 #   Column                                                                     Non-Null Count  Dtype  
---  ------                                                                     --------------  -----  
 0   (0, AG Tractors, Base)                                                     1881 non-null   float64
 1   (0, Combine Harvesting, Base)                                              41 non-null     float64
 2   (0, Commercial Mowing, Base)                                               2 non-null      float64
 3   (0, Compact 4WD Loaders, Base)                                             21 non-null     float64
 4   (0,

In [5]:
#start here
cust_purchase_history_date2 = pd.read_csv('associate_rule_with_date&category_PA.csv',index_col=0)
cust_purchase_history_date2
associate_rule_cust_date = cust_purchase_history_date2.iloc[1:,0:]
cust_purchase_history_date3 = cust_purchase_history_date2.iloc[1:,0:]
cust_purchase_history_date3
associate_rule_cust_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10699 entries, 5/20/19 to 10/29/20
Data columns (total 35 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   AG Tractors                                                1881 non-null   object
 1   Combine Harvesting                                         41 non-null     object
 2   Commercial Mowing                                          2 non-null      object
 3   Compact 4WD Loaders                                        21 non-null     object
 4   Compact Excavators                                         86 non-null     object
 5   Compact Track Loaders                                      67 non-null     object
 6   Excavators                                                 1 non-null      object
 7   Harvesters                                                 1 non-null      object
 8   Hay & Forage

In [6]:
for i in range(cust_purchase_history_date3.shape[0]):
    for j in range(cust_purchase_history_date3.shape[1]):
        if pd.isna(cust_purchase_history_date3.iloc[i][j]):
            associate_rule_cust_date.iloc[i][j] = 0
        else:
            associate_rule_cust_date.iloc[i][j] = 1
            

In [8]:
new_df_cust_date = associate_rule_cust_date
new_df_cust_date['Base'] = 0
new_df_cust_date['Implement'] = 0
new_df_cust_date = new_df_cust_date.astype('float') 
new_df_cust_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10699 entries, 5/20/19 to 10/29/20
Data columns (total 37 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   AG Tractors                                                10699 non-null  float64
 1   Combine Harvesting                                         10699 non-null  float64
 2   Commercial Mowing                                          10699 non-null  float64
 3   Compact 4WD Loaders                                        10699 non-null  float64
 4   Compact Excavators                                         10699 non-null  float64
 5   Compact Track Loaders                                      10699 non-null  float64
 6   Excavators                                                 10699 non-null  float64
 7   Harvesters                                                 10699 non-null  float64
 8   Ha

In [9]:
m = []
for i in range(new_df_cust_date.shape[0]):
    if (sum(new_df_cust_date.iloc[i,:21]) == 0):
        m.append(0)
    else:
        m.append(1)
n = []
for i in range(new_df_cust_date.shape[0]):
    if (sum(new_df_cust_date.iloc[i,21:]) == 0):
        n.append(0)
    else:
        n.append(1)
    
new_df_cust_date['Base'] = m
new_df_cust_date['Implement'] = n
new_df_cust_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10699 entries, 5/20/19 to 10/29/20
Data columns (total 37 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   AG Tractors                                                10699 non-null  float64
 1   Combine Harvesting                                         10699 non-null  float64
 2   Commercial Mowing                                          10699 non-null  float64
 3   Compact 4WD Loaders                                        10699 non-null  float64
 4   Compact Excavators                                         10699 non-null  float64
 5   Compact Track Loaders                                      10699 non-null  float64
 6   Excavators                                                 10699 non-null  float64
 7   Harvesters                                                 10699 non-null  float64
 8   Ha

In [10]:
new_df_cust_date.index = pd.to_datetime(new_df_cust_date.index)

In [12]:
PA_baseonly = new_df_cust_date.iloc[0:,21:36]
support_PA_baseonly = apriori(PA_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_PA = apriori(new_df_cust_date.iloc[0:,:35], min_support = 0.005, use_colnames = True, verbose = 1)
association_PA = association_rules(support_PA, metric = "confidence", min_threshold = 0.1)

Processing 42 combinations | Sampling itemset size 2
Processing 51 combinations | Sampling itemset size 3


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.036078,0.175811,0.022058,0.611399,3.477596,0.015715,2.120913
1,(AG Tractors),(Agriculture Implements),0.175811,0.036078,0.022058,0.125465,3.477596,0.015715,1.102211
2,(Cutters & Shredders),(AG Tractors),0.032059,0.175811,0.009253,0.288630,1.641706,0.003617,1.158594
3,(Landscape Equipment),(AG Tractors),0.037293,0.175811,0.008786,0.235589,1.340014,0.002229,1.078202
4,(Material Handling),(AG Tractors),0.070567,0.175811,0.023927,0.339073,1.928623,0.011521,1.247020
5,(AG Tractors),(Material Handling),0.175811,0.070567,0.023927,0.136098,1.928623,0.011521,1.075854
6,(Turf implements),(AG Tractors),0.088326,0.175811,0.013085,0.148148,0.842657,-0.002443,0.967526
7,(Snow Equipment),(Mowers),0.033741,0.464716,0.006356,0.188366,0.405335,-0.009324,0.659513
8,(Turf implements),(Mowers),0.088326,0.464716,0.029629,0.335450,0.721838,-0.011418,0.805483
9,(Agriculture Implements),(Material Handling),0.036078,0.070567,0.005141,0.142487,2.019164,0.002595,1.083870


In [13]:
support_PA_baseonly

,support,itemsets
0,0.036078,(Agriculture Implements)
1,0.032059,(Cutters & Shredders)
2,0.037293,(Landscape Equipment)
3,0.070567,(Material Handling)
4,0.033741,(Snow Equipment)
5,0.088326,(Turf implements)
6,0.814375,(Base)
7,0.022432,"(Agriculture Implements, Base)"
8,0.024582,"(Material Handling, Base)"
9,0.011029,"(Base, Snow Equipment)"


In [14]:
support_PA

,support,itemsets
0,0.175811,(AG Tractors)
1,0.008038,(Compact Excavators)
2,0.006262,(Compact Track Loaders)
3,0.029255,(Hay & Forage)
4,0.007290,(ISG)
5,0.464716,(Mowers)
6,0.006075,(Nutrient Application)
7,0.006169,(Planting & Seeding)
8,0.021030,(Portable Power Equipment)
9,0.008786,(Skid Steers)


In [15]:
association_PA

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.036078,0.175811,0.022058,0.611399,3.477596,0.015715,2.120913
1,(AG Tractors),(Agriculture Implements),0.175811,0.036078,0.022058,0.125465,3.477596,0.015715,1.102211
2,(Cutters & Shredders),(AG Tractors),0.032059,0.175811,0.009253,0.288630,1.641706,0.003617,1.158594
3,(Landscape Equipment),(AG Tractors),0.037293,0.175811,0.008786,0.235589,1.340014,0.002229,1.078202
4,(Material Handling),(AG Tractors),0.070567,0.175811,0.023927,0.339073,1.928623,0.011521,1.247020
5,(AG Tractors),(Material Handling),0.175811,0.070567,0.023927,0.136098,1.928623,0.011521,1.075854
6,(Turf implements),(AG Tractors),0.088326,0.175811,0.013085,0.148148,0.842657,-0.002443,0.967526
7,(Snow Equipment),(Mowers),0.033741,0.464716,0.006356,0.188366,0.405335,-0.009324,0.659513
8,(Turf implements),(Mowers),0.088326,0.464716,0.029629,0.335450,0.721838,-0.011418,0.805483
9,(Agriculture Implements),(Material Handling),0.036078,0.070567,0.005141,0.142487,2.019164,0.002595,1.083870


In [25]:
Jan1 = new_df_cust_date.loc['2019-01']
Jan2 = new_df_cust_date.loc['2020-01']
Jan3 = new_df_cust_date.loc['2021-01']
Jan = pd.concat([Jan1,Jan2,Jan3],axis=0)


Feb1 = new_df_cust_date.loc['2019-02']
Feb2 = new_df_cust_date.loc['2020-02']
Feb3 = new_df_cust_date.loc['2021-02']
Feb = pd.concat([Feb1,Feb2,Feb3],axis=0)

Mar1 = new_df_cust_date.loc['2019-03']
Mar2 = new_df_cust_date.loc['2020-03']
Mar3 = new_df_cust_date.loc['2021-03']
Mar = pd.concat([Mar1,Mar2,Mar3],axis=0)

Apr1 = new_df_cust_date.loc['2019-04']
Apr2 = new_df_cust_date.loc['2020-04']
Apr3 = new_df_cust_date.loc['2021-04']
Apr = pd.concat([Apr1,Apr2,Apr3],axis=0)


May1 = new_df_cust_date.loc['2019-05']
May2 = new_df_cust_date.loc['2020-05']
May3 = new_df_cust_date.loc['2021-05']
May = pd.concat([May1,May2,May3],axis=0)

Jun1 = new_df_cust_date.loc['2019-06']
Jun2 = new_df_cust_date.loc['2020-06']
Jun3 = new_df_cust_date.loc['2021-06']
Jun = pd.concat([Jun1,Jun2,Jun3],axis=0)

Jul1 = new_df_cust_date.loc['2019-07']
Jul2 = new_df_cust_date.loc['2020-07']
Jul3 = new_df_cust_date.loc['2021-07']
Jul = pd.concat([Jul1,Jul2,Jul3],axis=0)

Aug1 = new_df_cust_date.loc['2019-08']
Aug2 = new_df_cust_date.loc['2020-08']
Aug3 = new_df_cust_date.loc['2021-08']
Aug = pd.concat([Aug1,Aug2,Aug3],axis=0)

Sep1 = new_df_cust_date.loc['2019-09']
Sep2 = new_df_cust_date.loc['2020-09']
Sep3 = new_df_cust_date.loc['2021-09']
Sep = pd.concat([Sep1,Sep2,Sep3],axis=0)

Oct1 = new_df_cust_date.loc['2019-10']
Oct2 = new_df_cust_date.loc['2020-10']
Oct3 = new_df_cust_date.loc['2021-10']
Oct = pd.concat([Oct1,Oct2,Oct3],axis=0)

Nov1 = new_df_cust_date.loc['2019-11']
Nov2 = new_df_cust_date.loc['2020-11']
Nov3 = new_df_cust_date.loc['2021-11']
Nov = pd.concat([Nov1,Nov2,Nov3],axis=0)

Dec1 = new_df_cust_date.loc['2019-12']
Dec2 = new_df_cust_date.loc['2020-12']
Dec3 = new_df_cust_date.loc['2021-12']
Dec = pd.concat([Dec1,Dec2,Dec3],axis=0)

spring = pd.concat([Mar,Apr,May],axis=0)
summer = pd.concat([Jun,Jul,Aug],axis=0)
fall = pd.concat([Sep,Oct,Nov],axis=0)
winter = pd.concat([Dec,Jan,Feb],axis=0)

In [26]:
spring_baseonly = summer.iloc[0:,21:36]
support_spring_baseonly = apriori(spring_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_spring = apriori(spring.iloc[0:,:35], min_support = 0.005, use_colnames = True, verbose = 1)
association_spring = association_rules(support_spring, metric = "confidence", min_threshold = 0.1)

summer_baseonly = summer.iloc[0:,21:36]
support_summer_baseonly = apriori(summer_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_summer = apriori(summer.iloc[0:,:35], min_support = 0.005, use_colnames = True, verbose = 1)
association_summer = association_rules(support_summer, metric = "confidence", min_threshold = 0.1)

fall_baseonly = fall.iloc[0:,21:36]
support_fall_baseonly = apriori(fall_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_fall = apriori(fall.iloc[0:,:35], min_support = 0.005, use_colnames = True, verbose = 1)
association_fall = association_rules(support_fall, metric = "confidence", min_threshold = 0.1)

winter_baseonly = winter.iloc[0:,21:36]
support_winter_baseonly = apriori(winter_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_winter = apriori(winter.iloc[0:,:35], min_support = 0.005, use_colnames = True, verbose = 1)
association_winter = association_rules(support_winter, metric = "confidence", min_threshold = 0.1)

Processing 42 combinations | Sampling itemset size 2
Processing 51 combinations | Sampling itemset size 3
Processing 42 combinations | Sampling itemset size 2
Processing 30 combinations | Sampling itemset size 3
Processing 42 combinations | Sampling itemset size 2
Processing 54 combinations | Sampling itemset size 3
Processing 56 combinations | Sampling itemset size 2
Processing 8 combinations | Sampling itemset size 4


In [28]:
association_spring

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.036107,0.160928,0.024868,0.688742,4.279819,0.019058,2.695743
1,(AG Tractors),(Agriculture Implements),0.160928,0.036107,0.024868,0.154532,4.279819,0.019058,1.140070
2,(Cutters & Shredders),(AG Tractors),0.021760,0.160928,0.006934,0.318681,1.980275,0.003433,1.231541
3,(Landscape Equipment),(AG Tractors),0.039455,0.160928,0.008847,0.224242,1.393435,0.002498,1.081617
4,(Material Handling),(AG Tractors),0.060736,0.160928,0.023434,0.385827,2.397515,0.013660,1.366182
5,(AG Tractors),(Material Handling),0.160928,0.060736,0.023434,0.145617,2.397515,0.013660,1.099347
6,(Turf implements),(AG Tractors),0.081779,0.160928,0.013630,0.166667,1.035661,0.000469,1.006887
7,(Snow Equipment),(Mowers),0.014347,0.564323,0.005739,0.400000,0.708814,-0.002358,0.726128
8,(Turf implements),(Mowers),0.081779,0.564323,0.027260,0.333333,0.590678,-0.018890,0.653515
9,(Agriculture Implements),(Material Handling),0.036107,0.060736,0.005261,0.145695,2.398811,0.003068,1.099448


In [29]:
association_summer

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.030795,0.168271,0.017597,0.571429,3.395892,0.012415,1.940702
1,(AG Tractors),(Agriculture Implements),0.168271,0.030795,0.017597,0.104575,3.395892,0.012415,1.082397
2,(Cutters & Shredders),(AG Tractors),0.045092,0.168271,0.010173,0.225610,1.340756,0.002586,1.074044
3,(Landscape Equipment),(AG Tractors),0.036019,0.168271,0.007699,0.213740,1.270219,0.001638,1.057831
4,(Material Handling),(AG Tractors),0.066813,0.168271,0.023096,0.345679,2.054305,0.011853,1.271134
5,(AG Tractors),(Material Handling),0.168271,0.066813,0.023096,0.137255,2.054305,0.011853,1.081648
6,(Turf implements),(AG Tractors),0.078086,0.168271,0.010998,0.140845,0.837016,-0.002142,0.968079
7,(Turf implements),(Mowers),0.078086,0.484190,0.026670,0.341549,0.705403,-0.011138,0.783369


In [30]:
association_fall

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.042899,0.207536,0.024928,0.581081,2.799902,0.016025,1.891688
1,(AG Tractors),(Agriculture Implements),0.207536,0.042899,0.024928,0.120112,2.799902,0.016025,1.087753
2,(Cutters & Shredders),(AG Tractors),0.035362,0.207536,0.012754,0.360656,1.737797,0.005415,1.239495
3,(Landscape Equipment),(AG Tractors),0.036522,0.207536,0.011014,0.301587,1.453179,0.003435,1.134664
4,(Material Handling),(AG Tractors),0.087536,0.207536,0.030145,0.344371,1.659329,0.011978,1.208707
5,(AG Tractors),(Material Handling),0.207536,0.087536,0.030145,0.145251,1.659329,0.011978,1.067523
6,(Snow Equipment),(AG Tractors),0.063768,0.207536,0.007536,0.118182,0.569451,-0.005698,0.898670
7,(Turf implements),(AG Tractors),0.106667,0.207536,0.016812,0.157609,0.759427,-0.005326,0.940731
8,(Snow Equipment),(Mowers),0.063768,0.353043,0.009855,0.154545,0.437752,-0.012658,0.765217
9,(Turf implements),(Mowers),0.106667,0.353043,0.038261,0.358696,1.016010,0.000603,1.008814


In [31]:
association_winter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.042424,0.206061,0.021645,0.510204,2.475990,0.012903,1.620960
1,(AG Tractors),(Agriculture Implements),0.206061,0.042424,0.021645,0.105042,2.475990,0.012903,1.069967
2,(Cutters & Shredders),(AG Tractors),0.023377,0.206061,0.009524,0.407407,1.977124,0.004707,1.339773
3,(Landscape Equipment),(AG Tractors),0.034632,0.206061,0.008658,0.250000,1.213235,0.001522,1.058586
4,(Material Handling),(AG Tractors),0.092641,0.206061,0.019048,0.205607,0.997801,-0.000042,0.999430
5,(Turf implements),(AG Tractors),0.116883,0.206061,0.012121,0.103704,0.503268,-0.011964,0.885800
6,(Snow Equipment),(Mowers),0.132468,0.209524,0.013853,0.104575,0.499109,-0.013902,0.882795
7,(Turf implements),(Mowers),0.116883,0.209524,0.034632,0.296296,1.414141,0.010142,1.123308
8,(Mowers),(Turf implements),0.209524,0.116883,0.034632,0.165289,1.414141,0.010142,1.057992
9,(Agriculture Implements),(Material Handling),0.042424,0.092641,0.006061,0.142857,1.542056,0.002130,1.058586
